<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# DSI-SG-42 Project 3: Web APIs & NLP
### Reddit Scams: Are We Vulnerable?
---

## 2. Data Cleaning

### 2.1 Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import re
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### 2.2 Import scraped dataset

In [2]:
df = pd.read_csv('../data/cleaned_data_new.csv')
df.head()

,age,height,weight,bmi,yrssmok,packday,yrsquit,sleep_hours,health_status,phys_health_not_good,...,chd_mi,asthma_status,race_ethnicity,sex,education,income,smoker_status,e_cig_smoker,binge_drinker,heavy_drinker
0,80.0,NaN,NaN,NaN,0.0,0.0,0.0,8.0,Very Good,0 days,...,No,Never,White,Female,Grad College or Tech Sch,NaN,Never smoked,No,No,No
1,80.0,1.60,68.04,26.58,0.0,0.0,0.0,6.0,Excellent,0 days,...,No,Never,White,Female,Grad High Sch,$25k-$35k,Never smoked,No,No,No
2,56.0,1.57,63.50,25.76,0.0,0.0,0.0,5.0,Very Good,1-13 days,...,No,Never,White,Female,Grad College or Tech Sch,$100k-$200k,Never smoked,No,No,No
3,73.0,1.65,63.50,23.32,0.0,0.0,0.0,7.0,Excellent,0 days,...,No,Current,White,Female,Grad High Sch,NaN,Current smoker - some days,No,No,No
4,43.0,1.57,53.98,21.90,0.0,0.0,0.0,9.0,Fair,1-13 days,...,No,Never,White,Female,Attended College or Tech Sch,$25k-$35k,Never smoked,No,No,No


### 2.3 First look at data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445132 entries, 0 to 445131
Data columns (total 29 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   age                        445132 non-null  float64
 1   height                     416480 non-null  float64
 2   weight                     403054 non-null  float64
 3   bmi                        398564 non-null  float64
 4   yrssmok                    412104 non-null  float64
 5   packday                    412040 non-null  float64
 6   yrsquit                    409594 non-null  float64
 7   sleep_hours                439679 non-null  float64
 8   health_status              443934 non-null  object 
 9   phys_health_not_good       434205 non-null  object 
 10  mental_health_not_good     436065 non-null  object 
 11  last_routine_checkup       439333 non-null  object 
 12  visit_dentist_past_year    438111 non-null  object 
 13  health_insurance           42

### 2.7 Check for null values

In [4]:
df.isnull().sum()

age                              0
height                       28652
weight                       42078
bmi                          46568
yrssmok                      33028
packday                      33092
yrsquit                      35538
sleep_hours                   5453
health_status                 1198
phys_health_not_good         10927
mental_health_not_good        9067
last_routine_checkup          5799
visit_dentist_past_year       7021
health_insurance             17885
phy_exercise_past_30_days     1093
stroke                        1557
cancer                        2421
kidney_disease                1926
colon_sigmoidoscopy          22182
chd_mi                        5021
asthma_status                 3860
race_ethnicity               14057
sex                              0
education                     2383
income                       96047
smoker_status                35462
e_cig_smoker                 35660
binge_drinker                51102
heavy_drinker       

### 2.8 Initial Modelling (Baseline Scores only)

#### 2.8.0 LazyPredict to get Baseline scores

In [5]:
pip install lazypredict

Note: you may need to restart the kernel to use updated packages.


##### 2.8.0.1 Classifier Models

In [6]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

# Assuming 'df' is your DataFrame and 'target' is the name of the target column

X = df.drop('chd_mi', axis=1)
y = df['chd_mi']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# models is a DataFrame that contains the performance metrics for each model
print(models)

TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

##### 2.8.0.2 Regressor Models

In [ ]:
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split

# Assuming 'df' is your DataFrame and 'target' is the name of the target column for regression

X = df.drop('chd_mi', axis=1)
y = df['chd_mi']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit LazyRegressor
reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

# models is a DataFrame that contains the performance metrics for each model
print(models)

100%|██████████| 42/42 [02:52<00:00,  4.10s/it]

Empty DataFrame
Columns: [Adjusted R-Squared, R-Squared, RMSE, Time Taken]
Index: []


#### 2.8.1 XGBoost

We noticed that there were many comments/ replies that were auto-generated by bot. These do not serve much meaning to our analysis, hence we will remove these rows.

In [ ]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Drop rows where the target variable 'cvdcrhd4' is nan
df = df.dropna(subset=['chd_mi'])
# If you decide to remap, here is how you could do it for binary classification
mapping = {'No': 0, 'Yes': 1}
df['chd_mi'] = df['chd_mi'].map(mapping)

In [ ]:
# The column name in the dropna() method should match the column you're intending to use
df = df.dropna(subset=['chd_mi'])  # Make sure 'chd_mi' is the correct column name

In [ ]:
# Separate features and target
X = df.drop('chd_mi', axis=1)
y = df['chd_mi']

# Ensure y is of integer type
y = y.astype(int)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
# Convert object dtypes to category
for col in X_train.columns:
    if X_train[col].dtype == 'object':
        X_train[col] = X_train[col].astype('category')
        X_test[col] = X_test[col].astype('category')

In [ ]:
# Initialize the XGBoost classifier with enable_categorical=True
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', enable_categorical=True)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Accuracy: 91.19%


In [ ]:
# Train score
train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_pred)
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")

# Test score
test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, test_pred)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Preprocessing for CV score
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')

# Perform 5-fold cross-validation
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(model, X, y, cv=5)

# Print the average of the cross-validation scores
print(f"CV Mean Score: {cv_scores.mean() * 100:.2f}%")

# Optionally, you can also look at the individual folds' scores
print(f"CV Scores per Fold: {[f'{score * 100:.2f}%' for score in cv_scores]}")

Train Accuracy: 91.92%
Test Accuracy: 91.19%
CV Mean Score: 91.19%
CV Scores per Fold: ['91.18%', '91.19%', '91.22%', '91.19%', '91.18%']


#### 2.8.2 Random Forest

In [ ]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

# Separate features and target
X = df.drop('chd_mi', axis=1)
y = df['chd_mi']

# Impute missing values in features
imputer = SimpleImputer(strategy='most_frequent')
X_imputed = imputer.fit_transform(X)

# Convert back to DataFrame if needed
X_imputed = pd.DataFrame(X_imputed, columns=X.columns)

X_encoded = pd.get_dummies(X_imputed)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
# Initialize the Random Forest classifier
model = RandomForestClassifier(n_estimators=100, bootstrap=True, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
# Predictions for evaluation
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Train and Test accuracy scores
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Preprocessing for CV score
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')

# Perform 5-fold cross-validation to evaluate the model
cv_scores = cross_val_score(model, X_encoded, y, cv=5)

# Calculate the mean and standard deviation of the cross-validation scores
cv_mean = cv_scores.mean()
cv_std = cv_scores.std()
print(f"CV Mean Score: {cv_mean * 100:.2f}%")
print(f"CV Standard Deviation: {cv_std * 100:.2f}%")

Train Accuracy: 99.99%
Test Accuracy: 91.17%


#### 2.8.3 Decision Tree

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier

# Correctly create a preprocessing pipeline for numerical and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Add OneHotEncoder for categorical data
])

# Identify numeric and categorical columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Apply preprocessing
X_preprocessed = preprocessor.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, stratify=y, random_state=42)


In [ ]:
# Proceed with training and evaluating the Decision Tree model
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Preprocessing for CV score
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = X[col].astype('category')
        
cv_scores = cross_val_score(model, X_preprocessed, y, cv=5)
cv_mean = cv_scores.mean()
cv_std = cv_scores.std()
print(f"CV Mean Score: {cv_mean * 100:.2f}%")
print(f"CV Standard Deviation: {cv_std * 100:.2f}%")


Train Accuracy: 97.96%
Test Accuracy: 86.00%
CV Mean Score: 85.92%
CV Standard Deviation: 0.17%


### 2.10 Modelling (with parameters)

We will only run the models on df2. The reason being - when we dropped values, there were 0 rows for df1 - hence there is no point running model on an empty dataframe.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

#### Instantiating Pipeline

In [ ]:
# Logistic Regression Pipeline
logistic_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(max_iter=1000, random_state=42))
])

# SVM Pipeline
svm_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('model', SVC(random_state=42))
])

# XGBoost Pipeline
xgb_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    # No scaler needed for tree-based models
    ('model', xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42))
])

# Random Forest Pipeline
rf_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    # No scaler needed for tree-based models
    ('model', RandomForestClassifier(n_estimators=100, random_state=42))
])

#### Parameter Grids

In [ ]:
logistic_param_grid = {
    'model__C': [0.01, 0.1, 1, 10, 100],
    'model__penalty': ['l2'],
    'model__solver': ['saga'],
    'model__max_iter': [100, 1000, 5000]
}

In [ ]:
svm_param_grid = {
    'model__C': [1, 10],  # Reduced range of C values
    'model__gamma': ['scale', 0.01],  # Limited gamma to 'scale' and a representative value
    'model__kernel': ['rbf'],  # Focus on the RBF kernel, often the best choice for SVM
    # Removed 'model__degree': Typically, 'poly' kernel requires more processing time
}


In [ ]:
xgb_param_grid = {
    'model__max_depth': [5, 7],  # Focused on mid-range depths
    'model__n_estimators': [100, 200],  # Reduced the upper range
    'model__learning_rate': [0.1],  # Chose a commonly effective rate
    'model__subsample': [0.7, 1.0],  # Limited to higher subsampling for variance reduction
    'model__colsample_bytree': [0.7],  # Chose a moderate value for feature sampling
    'model__gamma': [0, 0.1]  # Simplified to two options to evaluate regularization benefit
}


In [ ]:
rf_param_grid = {
    'model__n_estimators': [100, 200],  # Reduced number of trees options
    'model__max_depth': [10, 20],  # Focus on moderate to high depth to control complexity
    'model__min_samples_split': [2, 5],  # Simplified range for minimum number of samples required to split
    'model__min_samples_leaf': [1, 2],  # Reduced range for the minimum number of samples required at a leaf node
    'model__max_features': ['auto'],  # Use the default option for the number of features to consider when looking for the best split
    'model__bootstrap': [True]  # Keep bootstrapping enabled for better generalization
    # Removed 'model__criterion': Simplification, sticking with the default 'gini' criterion
}


#### GridSearchCV Execution

In [ ]:
# Logistic Regression GridSearchCV
logistic_grid_search = GridSearchCV(logistic_pipeline, logistic_param_grid, cv=5, scoring='accuracy')
logistic_grid_search.fit(X_train, y_train)

# Best parameters and CV score
print("Best parameters for Logistic Regression:", logistic_grid_search.best_params_)
print("Best CV score for Logistic Regression:", logistic_grid_search.best_score_)

# Evaluate on training data using the best estimator found by GridSearchCV
y_train_pred = logistic_grid_search.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Train accuracy for Logistic Regression:", train_accuracy)

# Evaluate on test data using the best estimator found by GridSearchCV
y_test_pred = logistic_grid_search.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test accuracy for Logistic Regression:", test_accuracy)

# If you want to report the CV scores detail for the best model, you can do it by accessing cv_results_
# Here's how to get the mean CV score for the best estimator across folds
best_index = logistic_grid_search.best_index_
mean_cv_score = logistic_grid_search.cv_results_['mean_test_score'][best_index]
std_cv_score = logistic_grid_search.cv_results_['std_test_score'][best_index]
print(f"Mean CV score for the best Logistic Regression model: {mean_cv_score:.4f} ± {std_cv_score:.4f}")


Best parameters for Logistic Regression: {'model__C': 0.01, 'model__max_iter': 100, 'model__penalty': 'l2', 'model__solver': 'saga'}
Best CV score for Logistic Regression: 0.910648858406281
Train accuracy for Logistic Regression: 0.9106458502541889
Test accuracy for Logistic Regression: 0.9107546806564951
Mean CV score for the best Logistic Regression model: 0.9106 ± 0.0001


In [ ]:
# SVM GridSearchCV
svm_grid_search = GridSearchCV(svm_pipeline, svm_param_grid, cv=5, scoring='accuracy')
svm_grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best parameters for SVM:", svm_grid_search.best_params_)
print("Best score for SVM:", svm_grid_search.best_score_)

# Evaluate on training data using the best estimator found by GridSearchCV
y_train_pred = svm_grid_search.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Train accuracy for SVM", train_accuracy)

# Evaluate on test data using the best estimator found by GridSearchCV
y_test_pred = svm_grid_search.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test accuracy for SVM:", test_accuracy)

# If you want to report the CV scores detail for the best model, you can do it by accessing cv_results_
# Here's how to get the mean CV score for the best estimator across folds
best_index = svm_grid_search.best_index_
mean_cv_score = svm_grid_search.cv_results_['mean_test_score'][best_index]
std_cv_score = svm_grid_search.cv_results_['std_test_score'][best_index]
print(f"Mean CV score for the best SVM model: {mean_cv_score:.4f} ± {std_cv_score:.4f}")


NameError: name 'GridSearchCV' is not defined

In [ ]:
# XGBoost GridSearchCV
xgb_grid_search = GridSearchCV(xgb_pipeline, xgb_param_grid, cv=5, scoring='accuracy')
xgb_grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best parameters for XGBoost:", xgb_grid_search.best_params_)
print("Best score for XGBoost:", xgb_grid_search.best_score_)

# Evaluate on training data using the best estimator found by GridSearchCV
y_train_pred = xgb_grid_search.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Train accuracy for XGBoost:", train_accuracy)

# Evaluate on test data using the best estimator found by GridSearchCV
y_test_pred = xgb_grid_search.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test accuracy for XGBoost:", test_accuracy)

# If you want to report the CV scores detail for the best model, you can do it by accessing cv_results_
# Here's how to get the mean CV score for the best estimator across folds
best_index = xgb_grid_search.best_index_
mean_cv_score = xgb_grid_search.cv_results_['mean_test_score'][best_index]
std_cv_score = xgb_grid_search.cv_results_['std_test_score'][best_index]
print(f"Mean CV score for the best XGBoost model: {mean_cv_score:.4f} ± {std_cv_score:.4f}")


In [ ]:
# Random Forest GridSearchCV
rf_grid_search = GridSearchCV(rf_pipeline, rf_param_grid, cv=5, scoring='accuracy')
rf_grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best parameters for Random Forest:", rf_grid_search.best_params_)
print("Best score for Random Forest:", rf_grid_search.best_score_)

# Evaluate on training data using the best estimator found by GridSearchCV
y_train_pred = rf_grid_search.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Train accuracy for Random Forest:", train_accuracy)

# Evaluate on test data using the best estimator found by GridSearchCV
y_test_pred = rf_grid_search.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test accuracy for Random Forest:", test_accuracy)

# If you want to report the CV scores detail for the best model, you can do it by accessing cv_results_
# Here's how to get the mean CV score for the best estimator across folds
best_index = rf_grid_search.best_index_
mean_cv_score = rf_grid_search.cv_results_['mean_test_score'][best_index]
std_cv_score = rf_grid_search.cv_results_['std_test_score'][best_index]
print(f"Mean CV score for the best Random Forest model: {mean_cv_score:.4f} ± {std_cv_score:.4f}")


#### 2.9 Export data to CSV to proceed with EDA

In [ ]:
#df2.to_csv('../data/cleaned_data.csv', index=False)